<a href="https://colab.research.google.com/github/taheranjary/AppGAN/blob/main/AppGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Welcome to TecoGAN, an AI that will quadruple your video resolution using state of the art Generative-Adversarial-Networks.**

**Step 1:** 

Go to Edit>Notebook settings and make sure you have selected the run time as GPU. Click 'Save'
 
 
**Step 2:**

Run each code cell block below one by one, making sure that each completes before running the next one. The 'run' icon will appear if you hover your mouse over the very top-left corner of each code cell.

When you run the code cell with the command ***runMe()***, an upload prompt will appear asking you to upload the video you want to enhance. Go ahead and browse to the desired video from your PC and let the AI do its work. After completing the conversion, your enhanced video will automatically download. Note that your orginal and enhanced video file will be deleted from COLAB servers automatically. Do not worry.

You may close this tab once you are done.

**PS:**

If you want to convert more videos, just run the ***runME()** code cell again AFTER the previous conversion has completed and your file has been downloaded


**References**

[1]  Mengyu Chu, You Xie, Laura Leal-Taixe, Nils Thuerey. Learning Temporal Coherence via Self-Supervision for GAN-based
Video Generation. Technical University of Munich, 2020. https://arxiv.org/pdf/1811.09393.pdf

I take no credit for their work aside from tweaking their code to make it easier for others to use.


In [ ]:
from google.colab import files
import os
import time

filename = 'xxx'

!nvidia-smi
!git clone https://github.com/thunil/TecoGAN.git
%cd TecoGAN
!pip3 install -r requirements.txt
!mkdir /content/TecoGAN/LR/My_video
%cd /content/TecoGAN

In [ ]:
%%writefile runGan.py
'''
several running examples, run with
python3 runGan.py 1 # the last number is the run case number
runcase == 1    inference a trained model
runcase == 2    deleted
runcase == 3    deleted
runcase == 4    deleted
runcase == ...  
'''
import os, subprocess, sys, datetime, signal, shutil

runcase = int(sys.argv[1])
print ("Testing test case %d" % runcase)

def preexec(): # Don't forward signals.
    os.setpgrp()
    
def mycall(cmd, block=False):
    if not block:
        return subprocess.Popen(cmd)
    else:
        return subprocess.Popen(cmd, preexec_fn = preexec)
    
def folder_check(path):
    try_num = 1
    oripath = path[:-1] if path.endswith('/') else path
    while os.path.exists(path):
        print("Delete existing folder " + path + "?(Y/N)")
        decision = input()
        if decision == "Y":
            shutil.rmtree(path, ignore_errors=True)
            break
        else:
            path = oripath + "_%d/"%try_num
            try_num += 1
            print(path)
    
    return path

if( runcase == 0 ): # download inference data, trained models
    # download the trained model
    if(not os.path.exists("./model/")): os.mkdir("./model/")
    cmd1 = "wget https://ge.in.tum.de/download/data/TecoGAN/model.zip -O model/model.zip;"
    cmd1 += "unzip model/model.zip -d model; rm model/model.zip"
    subprocess.call(cmd1, shell=True)
    
elif( runcase == 1 ): # inference a trained model
    
    dirstr = './results/' # the place to save the results
    testpre = ['My_video'] # the test cases
    

    if (not os.path.exists(dirstr)): os.mkdir(dirstr)
    
    # run these test cases one by one:
    for nn in range(len(testpre)):
        cmd1 = ["python3", "main.py",
            "--cudaID", "0",            # set the cudaID here to use only one GPU
            "--output_dir",  dirstr,    # Set the place to put the results.
            "--summary_dir", os.path.join(dirstr, 'log/'), # Set the place to put the log. 
            "--mode","inference", 
            "--input_dir_LR", os.path.join("./LR/", testpre[nn]),   # the LR directory 
            #"--input_dir_HR", os.path.join("./HR/", testpre[nn]),  # the HR directory
            # one of (input_dir_HR,input_dir_LR) should be given
            "--output_pre", testpre[nn], # the subfolder to save current scene, optional
            "--num_resblock", "16",  # our model has 16 residual blocks, 
            # the pre-trained FRVSR and TecoGAN mini have 10 residual blocks
            "--checkpoint", './model/TecoGAN',  # the path of the trained model,
            "--output_ext", "png"               # png is more accurate, jpg is smaller
        ]
        mycall(cmd1).communicate()

elif( runcase == 2 ): # calculate all metrics, and save the csv files, should use png
    print('nunya')
    
elif( runcase == 3 ): # Train TecoGAN
    print('nunya')      
        
elif( runcase == 4 ): # Train FRVSR, loss = l2 warp + l2 content
    print('nunya')

Overwriting runGan.py


In [ ]:
!python3 runGan.py 0

def clear_reset():
  global filename
  %cd /content/sample_data
  if os.path.exists(filename):
    print('\n\nCUE deletion condition met\n\n')
    # Delete Content to reset
    # LR frames
    %cd /content/TecoGAN/LR
    !rm -rf My_video
    !mkdir /content/TecoGAN/LR/My_video

    # results HR frames
    %cd /content/TecoGAN/results
    !rm -rf My_video
    !mkdir /content/TecoGAN/results/My_video

    #orignal video input
    %cd /content/sample_data/
    cmdovi = 'rm ' + filename
    os.system(cmdovi)

    #final video output
    %cd /content/TecoGAN
    !rm output4R.mp4
  else:
    print('\n\nCUE Deletion conition not met\n\n')


def infer():
  global filename

  # Upload Video Prompt
  %cd /content/sample_data
  uploaded = files.upload()

  # extract and save LR frames
  filename = next(iter(uploaded))
  cmdf = 'ffmpeg -i /content/sample_data/' + filename + ' %04d.png'
  %cd /content/TecoGAN/LR/My_video
  #!ffmpeg  -i /content/sample_data/slowmfirewire.MOV %04d.png
  os.system(cmdf)

  # TecoGAN generate and save HR frames
  %cd /content/TecoGAN
  %tensorflow_version 1.x
  !python3 runGan.py 1

  # convert HR frames to mp4 file
  #!ffmpeg -vsync 0 -hwaccel cuvid -c:v mjpeg_cuvid -framerate 24.00 -i /content/TecoGAN/results/My_video/*.png -c:v h264_nvenc quaid2.mp4
  !ffmpeg -f image2 -framerate 30 -i /content/TecoGAN/results/My_video/output_%04d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p output4R.mp4

  # Download HR mp4 File
  %cd /content/TecoGAN
  files.download('output4R.mp4')

def runMe():
  clear_reset()
  infer()

In [ ]:
runMe()

In [ ]:
# %cd /content
# !rm -rf TecoGAN

In [ ]:
#%cd /content/TecoGAN